# Mitigating Overfitting Using Regularization to Defend Networks Against Adversarial Examples

We will write supplemantal explations and show additional analysis for our experiments.

# Regularizers in our experiments
In this section, we briefly outline the regularizers that we used for the experiments.
## Dropout
Dropout [8] is a technique for avoiding ovefitting by randomly dropping the contribution of units from the networks during the training. The idea is that units do not specialize to specific training examples.  Units are usually not dropped out during test time. It is common to use a  dropout probability of 50 $\%$. In our case, we changed this probability from 10, 30, 50, 70, and 90 $\%$.

## L1 and L2 weight decay
L1 and L2 weight decay are regularizers for the networks in form of weight decay. They can be derived from adding a penalty term to the loss function that represet a prior. A common choice is to encourage small weights with a Gaussian or Laplacian prior around zero. The corresponding L1 and L2 weight decay is described by add the penalty to the loss function $ \rm Loss(w; x, y) $ as follows:

\begin{equation}
  \tilde{ \rm  Loss(w; x,y)} = \rm Loss(w; x, y) + \lambda||w||^n,
  \label{eq:weight_decay}
\end{equation}

where $ n $ is the norm size,  $ \lambda $ is the regularization parameter that determines the regularization strength, and $ w $ are the weights of the networks. In our study, we changed the regularization parameter from 1e-5, 1e-4, 1e-3, 1e-2, and 1e-1 to test how a range of weak to strong regularizers work on adversarial examples.

## Binarized Stochastic Neural Networks
Binarized Stochastic Neural Networks (BSN) are networks with binarized activations of neurons (1 or 0). This binarization serves as a regularizer [1] at it prevents some ability of finetuning individual weights. These binarized neurons have activation $h$ set to a value of 0 or 1. This is described by the probability $p$ of activation of a neuron in the networks to be 1. The activation is zero otherwise. This could be implemented by $ p = \sigma (a) $ where $ a $ is the input, and $ \sigma $ is the logistic activation function. We therefore define the output of the binarized neuron as:

\begin{equation}
  h = 1_{z < \sigma (a)},
  \label{eq:BSN}
\end{equation}

where $ 1_z $ is the indicator function with uniform distribution $ z \sim U[0,1] $.
This function is non-differentiable to backpropagate the gradient. To overcome this problem, there have been several proposals for how to approximate and propagate gradients through stochastic neurons [1] [4]. One suggestion, introduced by [1], is a special case of the REINFORCE algorithm [5] which uses a sampling technique. The algorithm estimates the expectation of the gradient of a loss function with respect to the inputs with a baseline as:

\begin{equation}
  \mathbb{E}[\frac{\partial Loss(w;x,y) }{\partial a} ] = \mathbb{E}[(h - \sigma (a)) ( Loss(w;x,y)-\bar{ Loss(w;x,y)}) ],
  \label{eq:RF_1}
\end{equation}

where $ \bar{ \rm Loss(w;x,y)} $ is a baseline as:

\begin{equation}
  \bar{ \rm Loss(w;x,y)} = \frac{ \mathbb{E}[(h - \sigma (a))^2 \rm Loss(w;x,y)]}{ \mathbb{E}[(h - \sigma (a))^2] }.
  \label{eq:RF_2}
\end{equation}

This baseline is to minimize the variance of the estimation. For this estimator, [1] note how the estimator requires broadcasting $ \rm {Loss(w;x,y)} $ throughout the networks.

The other proposals are called straight-through estimators (STs) [1] [4]. One suggestion [4] is to use the gradient of the logistic function for the backpropagation. In our case, we use STE with the slope-annealing trick [2] to gradually increase the slope of the logistic function. This slope reduces the discrepancy between the two functions used during the forward and backward pass. The slope-annealing trick modify the $ h $  with multiplying the input $ a $ by the slope m as follows:

\begin{equation}
  h = 1_{z < \sigma (ma)}.
  \label{eq:BSN_slope}
\end{equation}
The slope is increased gradually while training the networks. The computation of the gradient of the output with respect to the input will be

\begin{equation}
  \frac{\partial h}{\partial a} = \frac{\partial \sigma(ma)}{\partial a}.
  \label{eq:gradient_slope}
\end{equation}
In this study we use the STE with a slope-annealing technique for the dense layer in the network. 




# Specific parameter setting
We start by applying a shallow convolutional neural network (CNN) to the MNIST data with changing dropout probabilities. This network consists of one convolutional layer with kernels 64 filters and each filter size 3$\times$3 and stride 2, and two dense layers with node sizes 100 and 10. All of these dense layers are deterministic. We use dropout with the dropout probability 10, 30, 50, 70, 90, and also compare this to a network without any regularizations. 

The second experiment is the same as the first, but we use the $ L1 $ and $ L2 $ weight decay instead of dropout. The network condition is the same as the first one. The hyperparameter $ \lambda $  are 1e-5, 1e-4, 1e-3, 1e-2, and 1e-1. As before, we compare the results to the network without any regularizations as a baseline for this experiment.

Finally, we applied the binarized stochastic MLP (BSN), both with the Straight through estimator (ST-MLP) and Reinforce algorithm (REINFORCE-MLP) and a binarized deterministic MLP (BDN-MLP) which are also combined with the convolutional layer(s) using the two estimator methods. The MLP architectures consist of one hidden layer with 100 nodes. The CNN is the same as the architecture for the first experiments, but the dense layers are stochastic and deterministic binarized neurons. We additionally applied 2 convolutional layers CNN (2CNN) to the dataset adding one more convolutional layer with 64 filters and each filter size 3$\times$3 to 1 convolutional layer CNN (1CNN) to test if we can improve the accuracies on adversarial examples. For the slope techniques in ST we increased the slope $m$ with the following schedule $ m = min(5, 1 + 0.04 * N_{epoch} )$ where $N_{epoch}$ is the number of epoch. For BDN, we round the output of sigmoid activation function for the output $ h =  round(\sigma(a)) $. 

All training contains clean images. On testing, we use adversarial examples (white-box) and do not use adversarial training [3]. All activation units for convolutional layers are rectifier linear units and MLP, but for the first dense layer of BSN and BDN, we use the logistic activation function. We used the softmax cross entropy loss function for training. The learning rate for the two first experiments are 1e-4. For the stochastic neuron experiment, the learning rates for MLPs, 1CNNs, and 2CNNs of both BDN and BSN are 1e-2, 1e-3, and 1e-3, respectively. The training epoch sizes for 1st and 2nd experiments are 20 epochs, and the stochastic neuron experiment, the epoch sizes are 20, 30, and 30 for MLPs, 1CNNs, and 2CNNs of both BSN and BDN, respectively. We use the Adam optimizer [6] and Xaiver initialization [7] for all of the networks. All networks are implemented in Tensorflow. We used a GPU GeForce 780 and ran each model 10 times independently. The results are averaged over the 10 runs. 


# Confidence level for each experiment
Here, we show the confidence level figures for each experiments in the paper.

## Confidence level on changing dropout probability experiment

<img src="conf/wo_rg_conf.png">
<img src="conf/dp_10_conf.png">
<img src="conf/dp_50_conf.png">
<img src="conf/dp_90_conf.png">

Proportion (0 to 1) vs confidence level (0-100) for target classes of adversarial examples on changing dropout probability experiments (without any regularizations, 10, 50, and 90 $\%$ probabilities). $\epsilon$ means perturbation level (0.01 to 0.25). Proportion 0 means none of the examples, and 1 means all of the examples. Also, Confidence level 0 means 0 $\%$ confidence for the target class, and 100 means 100 $\%$ confidence for the target class. Dropout probability of 50 and 90 $\%$ gradually change confidence level from high to low while increasing the perturbation level. However, obviously, confidence levels for the lower probability and without any regularizations rapidly change from high to low.

## Confidence level on Changing L1 and L2  $\lambda$ parameters 

### L1 weight Decay

<img src="conf/wo_rg_conf.png">
<img src="conf/l1_lambda5_conf.png">
<img src="conf/l1_lambda2_conf.png">
<img src="conf/l1_lambda1_conf.png">

Proportion (0 to 1) vs confidence level (0-100) for target classes of adversarial examples on changing L1 $ \lambda $ experiments (without any regularizations, $\lambda = 1e-5, 1e-2, 1e-1 $). The confidence levels for W/O any regularizations and $\lambda = 1e-5 $ change very similarly. At $ \epsilon = 0.15 $, most of both confidence levels are zeros. On the other hand, confidence levels for $\lambda = 1e-2, 1e-1 $ gradually change from high to low.

### L2 weight decay

<img src="conf/wo_rg_conf.png">
<img src="conf/l2_lambda5_conf.png">
<img src="conf/l2_lambda2_conf.png">
<img src="conf/l2_lambda1_conf.png">

Proportion (0 to 1) vs confidence level (0-100) for target classes of adversarial examples on changing L2 $ \lambda $ experiments (without any regularizations, $\lambda = 1e-5, 1e-2, 1e-1 $). These results are similar to L1. The high $ \lambda $ gradually change the confidence levels from high to lower, but the lower change the confidence levels rapidly.

### Confidence level on Binarized Stochastic Neural Networks

#### MLP

<img src="conf/dt_mlp_conf.png">
<img src="conf/st_mlp_conf.png">
<img src="conf/rf_mlp_conf.png">

Proportion (0 to 1) vs confidence level (0-100) for target classes on binarized stochastic and deterministic experiments for MLPs. Most of confidence levels for BDN-MLP at $\epsilon = 0.15$ are zeros. But some of the confidence levels for ST and REINFORCE MLPs at $\epsilon = 0.25$ are still 100 $\%$, especially REINFORCE-MLP.

#### 1CNN

<img src="conf/dt_1cnn_conf2.png">
<img src="conf/st_1cnn_conf2.png">
<img src="conf/rf_1cnn_conf2.png">

Proportion (0 to 1) vs confidence level (0-100) for target classes on binarized stochastic and deterministic experiments for 1CNN. BDN and ST 1CNNs have very similar behaviors. Most of the confidence levels at lower $\epsilon $ are 100 $\%$. while increasing $\epsilon $, the proportions for BDN and ST are divided into two, close to 0 or 100$\%$, even at $\epsilon = 0.25$. The proportions of middle of confidence levels are less. On the other hand, REINOFCE-1CNN are more robust than the others. The proportions of high confidence levels for REINFORCE gradually decrease but not too much.

#### 2CNN

<img src="conf/dt_2cnn_conf2.png">
<img src="conf/st_2cnn_conf2.png">
<img src="conf/rf_2cnn_conf2.png">

Proportion (0 to 1) vs confidence level (0-100) for target classes on binarized stochastic and deterministic experiments for 2CNN. Similar to 1CNN result, BDNs and ST 2CNNs are very similar.The proportions for REINFORCE in high confidence levels are more robust than the others in this experiment too.

# References


1. Bengio, Y., Léonard, N., Courville, A.C.: Estimating or propagating gradients through stochastic neurons for conditional computation. arXiv preprint arXiv:1308.3432 (2013)
2. Chung, J., Ahn, S., Bengio, Y.: Hierarchical multiscale recurrent neural networks. arXiv preprint arXiv:1609.01704 (2016)
3. Goodfellow, I.J., Shlens, J., Szegedy, C.: Explaining and harnessing adversarial examples. arXiv preprint arXiv:1412.6572 (2014)
4. Hinton, G.: Lecture 9.3 – Using noise as a regularizer. COURSERA: Neural Networks for Machine Learning (2012)
5. Williams, R.J.: Simple statistical gradient-following algorithms for connectionist reinforcement learning. Machine Learning 8, 229–256 (1992)
6. Kingma, D.P., Ba, J.: Adam: A method for stochastic optimization. arXiv preprint arXiv:1412.6980 (2014)
7. Glorot, X., Bengio, Y.: Understanding the difficulty of training deep feedforward neural networks. In: In Proceedings of the International Conference on Artificial Intelligence and Statistics (AISTATS’10). Society for Artificial Intelligence and Statistics (2010)
8. Srivastava, N., Hinton, G.E., Krizhevsky, A., Sutskever, I., Salakhutdinov, R.: Dropout: a simple way to prevent neural networks from overfitting. Journal of Machine Learning Research 15(1), 1929–1958 (2014), http://www.cs.toronto.
edu/~rsalakhu/papers/srivastava14a.pdf







